# Dhruv Rana #

#### Remove Previous Videos ####

In [1]:
import os
import shutil

if os.path.exists("video"): 
    shutil.rmtree("video")

#### Import Modules ####

In [2]:
import gym
from gym.wrappers import RecordVideo

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


#### Run Env. w/o Training ####

In [5]:
env = gym.make("CartPole-v1",render_mode='rgb_array')
env = RecordVideo(env,video_folder="./video",video_length=2000,name_prefix="No_Training_Episode0",)

env.reset()
for i in range(30):

  # env.action_space.sample() produces either 0 (left) or 1 (right).
  output = env.step(env.action_space.sample())

  observation = output[0] # Cart Position , Cart Velocity, Pole Angle, Pole Angular Velocity
  reward = output[1] # +1 for every step (Goal : keep the pole up as long as possible)
  done = output[2] # boolean : True-> when pole falls (pole angle more than 12, postioion more than 2.4)
  info = output[3] # 
  
  print("step", i, observation, reward, done, info)

env.close()

c:\Users\dhruv\.conda\envs\ComputerVision\lib\site-packages\gym\envs\classic_control\cartpole.py:177: UserWarning: WARN: You are calling 'step()' even though this environment has already returned terminated = True. You should always call 'reset()' once you receive 'terminated = True' -- any further steps are undefined behavior.
  logger.warn(


step 0 [ 0.00184955  0.20331004  0.03174967 -0.31426242] 1.0 False False
step 1 [ 0.00591575  0.00775053  0.02546442 -0.01173817] 1.0 False False
step 2 [ 0.00607077  0.20249821  0.02522966 -0.29627925] 1.0 False False
step 3 [0.01012073 0.00702584 0.01930407 0.00425272] 1.0 False False
step 4 [ 0.01026125  0.2018657   0.01938913 -0.28227758] 1.0 False False
step 5 [0.01429856 0.00647263 0.01374357 0.01645698] 1.0 False False
step 6 [ 0.01442801 -0.1888437   0.01407271  0.3134443 ] 1.0 False False
step 7 [ 0.01065114 -0.38416326  0.0203416   0.61053187] 1.0 False False
step 8 [ 0.00296787 -0.57956356  0.03255224  0.9095517 ] 1.0 False False
step 9 [-0.0086234  -0.7751106   0.05074327  1.2122855 ] 1.0 False False
step 10 [-0.02412561 -0.580679    0.07498898  0.93592566] 1.0 False False
step 11 [-0.03573919 -0.77672786  0.09370749  1.2511985 ] 1.0 False False
step 12 [-0.05127374 -0.97291726  0.11873147  1.5717013 ] 1.0 False False
step 13 [-0.07073209 -0.77939534  0.15016548  1.3182871 

Moviepy - Done !
Moviepy - video ready c:\Users\dhruv\Desktop\College\Rutgers\2022FA(Fall)\ROBOTICS&COMP VISION\Robotic-Code-on-Git\Project 2\video\No_Training_Episode0-episode-0.mp4


In [6]:
from base64 import b64encode

def play_video(filename):
    video = open(filename,'rb').read()
    decode = 'data:video/mp4;base64,' + b64encode(video).decode()
    return decode

#### Display Video ####

In [7]:
from IPython.display import HTML

HTML("""
<video controls>
    <source src = "%s" type ="video/mp4">
</video>
"""%play_video("video/No_Training_Episode0-episode-0.mp4")
)

#### Create Deep Netwrok ####

In [8]:
class DQN(nn.Module):
    def __init__(self, h, w, outputs):
        super(DQN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=2)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=2)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=5, stride=2)
        self.bn3 = nn.BatchNorm2d(32)

        # Number of Linear input connections depends on output of conv2d layers
        # and therefore the input image size, so compute it.
        def conv2d_size_out(size, kernel_size = 5, stride = 2):
            return (size - (kernel_size - 1) - 1) // stride  + 1
        convw = conv2d_size_out(conv2d_size_out(conv2d_size_out(w)))
        convh = conv2d_size_out(conv2d_size_out(conv2d_size_out(h)))
        linear_input_size = convw * convh * 32
        self.head = nn.Linear(linear_input_size, outputs)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = x.to(device)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        return self.head(x.view(x.size(0), -1))